# Overview
- Functools
- itertools

## Functools
- functions that act on or return other functions
    - reduce()
    - generic function (multiple functions implementing the same operation for different types)
        - PEP443
        -  @singledispatchmethod for classes
    - cache and lru_cache decorator for cpu-intense computation that do not change often (memoization)
    - total_ordering - eq() and lt() or gt() etc. -> rest is computed
    - wraps to create own decorator
        - options to pass params to decorator function (not decorated function) (no example)
        - option to stack params (no example)

In [10]:
import functools
lisi = [1,2,3,4,5]

#def reduce(function, iterable, initializer=None):
# calculates ((((1+2)+3)+4)+5). 
# The left argument, x, is the accumulated value and 
# the right argument, y, is the update value from the iterable
print(functools.reduce(lambda x, y: x+y, lisi))
print(functools.reduce(lambda x, y: x+y, lisi, 5))

15
20


In [17]:
from functools import singledispatch

@singledispatch
def fun(arg, verbose=False):
    if verbose:
        print("Let me just say,", end=" ")
    print(arg)

@fun.register
def _(arg: int, verbose=False):
   if verbose:
       print("Strength in numbers, eh?", end=" ")
   print(arg)

@fun.register
def _(arg: list, verbose=False):
    if verbose:
        print("Enumerate this:")
    for i, elem in enumerate(arg):
        print(i, elem)

fun('hello', verbose=True)
fun(10, verbose=True)
fun([1,'huhu', '20'],verbose=True)


Let me just say, hello
Strength in numbers, eh? 10
Enumerate this:
0 1
1 huhu
2 20


In [3]:
# Memoization
# maxsize = num function calls
# cache decorator is smaller and faster
from functools import lru_cache, cache
import time

def count_vowels_no_cache(sentence: str):
    sentence = sentence.casefold()
    return sum(sentence.count(vowel) for vowel in 'aeiou')

@lru_cache(maxsize=100)
def count_vowels_lru(sentence: str):
    sentence = sentence.casefold()
    return sum(sentence.count(vowel) for vowel in 'aeiou')

@cache
def count_vowels_cache(sentence: str):
    sentence = sentence.casefold()
    return sum(sentence.count(vowel) for vowel in 'aeiou')

begin = time.time()
print(count_vowels_no_cache("This is a nice caching function!"))
end = time.time()
print("Time taken to count vowels without caching: ", end-begin)

begin = time.time()
print(count_vowels_lru("This is a nice caching function!"))
end = time.time()
print("Time taken to count vowels with caching lru 1st run: ", end-begin)

begin = time.time()
print(count_vowels_lru("This is a nice caching function!"))
end = time.time()
print("Time taken to count vowels with caching lru 2st run: ", end-begin)

begin = time.time()
print(count_vowels_cache("This is a nice caching function!"))
end = time.time()
print("Time taken to count vowels with caching 1st run: ", end-begin)

begin = time.time()
print(count_vowels_cache("This is a nice caching function!"))
end = time.time()
print("Time taken to count vowels with caching 1st run: ", end-begin)

10
Time taken to count vowels without caching:  0.0004470348358154297
10
Time taken to count vowels with caching lru 1st run:  0.004292964935302734
10
Time taken to count vowels with caching lru 2st run:  0.00013303756713867188
10
Time taken to count vowels with caching 1st run:  0.00022602081298828125
10
Time taken to count vowels with caching 1st run:  0.000225067138671875


In [7]:
# Total Ordering
# Performance impact! define all 6 manually if to slow
# eq() and lt() defined manually
# => python computes gt(), ge(), le(), noteq() 
from functools import total_ordering

@total_ordering
class Actor:
    def __init__(self, lastname, firstname):
        self.firstname = firstname
        self.lastname = lastname
    def __eq__(self, other):
        return ((self.lastname.lower(), self.firstname.lower()) ==
                (other.lastname.lower(), other.firstname.lower()))
    def __lt__(self, other):
        return ((self.lastname.lower(), self.firstname.lower()) <
                (other.lastname.lower(), other.firstname.lower()))


actor1 = Actor("Alfred","James")
actor2 = Actor("Haddock","Captain")

# Auto computed
print(actor2 > actor1) 
print(actor2 >= actor1)
print(actor2 != actor1)

True
True
True


In [31]:
# Wraps
# 1. Decorator is called and returns wrapper
# 2. Wrapper function is called, decorates + passes args to wrapped function
# 3. Inner function is called and result returned
from functools import wraps
import time

# timing = decorator
def timing(fn):
    print("Decorator timing begin...")
    @wraps(fn)
    # args and kwargs passed on to decorated function 
    def wrapper(*args, **kwargs):
        print("Wrapper timing begin...")
        start_time = time.perf_counter()
        # inner function called
        fn_result = fn(*args, **kwargs)
        end_time = time.perf_counter()
        time_duration = end_time - start_time
        print("Function {} took: {} s".format(fn.__name__, time_duration))
        print("Wrapper timing end...")
        return fn_result
    print("Decorator timing end...")
    return wrapper

def fn_no_params():
    return "gugugag"

def fn_with_params(a, b, c=None):
    return a + b if c else 0

@timing
def fn_with_params_dec(a, b, c=None):
    return a + b if c else 0


# No need for params, function could be passed directly
# Wrapper object is returned
decorated_fun = timing(fn_no_params)

# Wrapper object is called now
print(decorated_fun())

# Works thanks to *args and **kwargs
decorated_fun2 = timing(fn_with_params)
print(decorated_fun2(a=10, b=20, c=True))

# Decorator style
print(fn_with_params_dec(a=10, b=20, c=True))



Decorator timing begin...
Decorator timing end...
Decorator timing begin...
Decorator timing end...
Wrapper timing begin...
Function fn_no_params took: 1.213999894389417e-06 s
Wrapper timing end...
gugugag
Decorator timing begin...
Decorator timing end...
Wrapper timing begin...
Function fn_with_params took: 2.41899988395744e-06 s
Wrapper timing end...
30
Wrapper timing begin...
Function fn_with_params_dec took: 1.725999936752487e-06 s
Wrapper timing end...
30
